In [2]:
import wandb
! pip install torch
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import re
import time
from collections import Counter
from datasets import load_dataset
from snorkel.labeling import labeling_function, PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter


In [4]:
!pip install snorkel

In [4]:
# ====================================================================
# Q4: Sequential Training with W&B (CIFAR 10/100) (COMPLETED)
# ====================================================================
print("\n--- Starting Q4: Sequential CIFAR Training ---")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS_PER_STAGE = 100
BATCH_SIZE = 128

# Simple CNN Model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10): # Corrected the spelling of __init__
        super(SimpleCNN, self).__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2), # 32x16x16
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2)  # 64x8x8
        )
        self.fc = nn.Linear(64 * 8 * 8, num_classes)

    def forward(self, x):
        x = self.conv_stack(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def update_num_classes(self, new_num_classes):
        in_features = self.fc.in_features
        self.fc = nn.Linear(in_features, new_num_classes)
        return self # Return self to allow chaining .to(device)

# Data Transformations and Loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainloader_10 = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform),
    batch_size=BATCH_SIZE, shuffle=True
)
trainloader_100 = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform),
    batch_size=BATCH_SIZE, shuffle=True
)

# Training Function
def train_model_stage(model, dataloader, criterion, optimizer, num_epochs, task_name, log_offset):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(dataloader)
        epoch_acc = 100 * correct / total

        wandb.log({
            "epoch": epoch + 1 + log_offset,
            f"{task_name}_loss": epoch_loss,
            f"{task_name}_accuracy": epoch_acc,
        })
        print(f"Epoch {epoch+1+log_offset} ({task_name}): Loss={epoch_loss:.4f}, Acc={epoch_acc:.2f}%")
    return model

# --- Experiment (a): CIFAR 100 then CIFAR 10 ---
wandb.init(project="Q4-cifar-experiments", name="CIFAR100_then_10")

# Stage 1: C100 (100 classes)
model_a = SimpleCNN(num_classes=100).to(device)
criterion_a = nn.CrossEntropyLoss()
optimizer_a = optim.Adam(model_a.parameters(), lr=0.001)

model_a = train_model_stage(model_a, trainloader_100, criterion_a, optimizer_a,
                            num_epochs=EPOCHS_PER_STAGE, task_name="C100_S1", log_offset=0)

# Stage 2: C10 (10 classes)
model_a = model_a.update_num_classes(10).to(device)
optimizer_a = optim.Adam(model_a.parameters(), lr=0.001) # Reset optimizer
model_a = train_model_stage(model_a, trainloader_10, criterion_a, optimizer_a,
                            num_epochs=EPOCHS_PER_STAGE, task_name="C10_S2", log_offset=EPOCHS_PER_STAGE)

wandb.finish()


# --- Experiment (b): CIFAR 10 then CIFAR 100 ---
wandb.init(project="Q4-cifar-experiments", name="CIFAR10_then_100")

# Stage 1: C10 (10 classes)
model_b = SimpleCNN(num_classes=10).to(device)
criterion_b = nn.CrossEntropyLoss()
optimizer_b = optim.Adam(model_b.parameters(), lr=0.001)

model_b = train_model_stage(model_b, trainloader_10, criterion_b, optimizer_b,
                            num_epochs=EPOCHS_PER_STAGE, task_name="C10_S1", log_offset=0)

# Stage 2: C100 (100 classes)
model_b = model_b.update_num_classes(100).to(device)
optimizer_b = optim.Adam(model_b.parameters(), lr=0.001) # Reset optimizer
model_b = train_model_stage(model_b, trainloader_100, criterion_b, optimizer_b,
                            num_epochs=EPOCHS_PER_STAGE, task_name="C100_S2", log_offset=EPOCHS_PER_STAGE)

wandb.finish()
print("\nQ4 finished. Two sequential runs logged to W&B.")


--- Starting Q4: Sequential CIFAR Training ---


Epoch 1 (C100_S1): Loss=3.3902, Acc=21.11%
Epoch 2 (C100_S1): Loss=2.6052, Acc=35.81%
Epoch 3 (C100_S1): Loss=2.2242, Acc=43.97%
Epoch 4 (C100_S1): Loss=1.9383, Acc=50.42%
Epoch 5 (C100_S1): Loss=1.6843, Acc=56.04%
Epoch 6 (C100_S1): Loss=1.4660, Acc=61.35%
Epoch 7 (C100_S1): Loss=1.2581, Acc=66.22%
Epoch 8 (C100_S1): Loss=1.0673, Acc=71.22%
Epoch 9 (C100_S1): Loss=0.9034, Acc=75.45%
Epoch 10 (C100_S1): Loss=0.7631, Acc=79.17%
Epoch 11 (C100_S1): Loss=0.6359, Acc=82.29%
Epoch 12 (C100_S1): Loss=0.5241, Acc=85.47%
Epoch 13 (C100_S1): Loss=0.4313, Acc=87.98%
Epoch 14 (C100_S1): Loss=0.3443, Acc=90.48%
Epoch 15 (C100_S1): Loss=0.2842, Acc=92.19%
Epoch 16 (C100_S1): Loss=0.2364, Acc=93.50%
Epoch 17 (C100_S1): Loss=0.2016, Acc=94.31%
Epoch 18 (C100_S1): Loss=0.1861, Acc=94.65%
Epoch 19 (C100_S1): Loss=0.1645, Acc=95.22%
Epoch 20 (C100_S1): Loss=0.1379, Acc=96.13%
Epoch 21 (C100_S1): Loss=0.1303, Acc=96.30%
Epoch 22 (C100_S1): Loss=0.1160, Acc=96.69%
Epoch 23 (C100_S1): Loss=0.1122, Acc=96.7

C100_S1_accuracy,▁▅▅▆▇▇▇█████████████████████████████████
C100_S1_loss,█▆▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
C10_S2_accuracy,▁▃▃▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████████████
C10_S2_loss,█▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
C100_S1_accuracy,98.884
C100_S1_loss,0.03954
C10_S2_accuracy,98.932
C10_S2_loss,0.03312
epoch,200


Epoch 1 (C10_S1): Loss=1.4351, Acc=49.32%
Epoch 2 (C10_S1): Loss=1.0912, Acc=62.15%
Epoch 3 (C10_S1): Loss=0.9557, Acc=66.95%
Epoch 4 (C10_S1): Loss=0.8687, Acc=70.09%
Epoch 5 (C10_S1): Loss=0.8034, Acc=72.44%
Epoch 6 (C10_S1): Loss=0.7528, Acc=74.22%
Epoch 7 (C10_S1): Loss=0.7086, Acc=75.63%
Epoch 8 (C10_S1): Loss=0.6712, Acc=77.01%
Epoch 9 (C10_S1): Loss=0.6363, Acc=78.16%
Epoch 10 (C10_S1): Loss=0.6033, Acc=79.36%
Epoch 11 (C10_S1): Loss=0.5763, Acc=80.37%
Epoch 12 (C10_S1): Loss=0.5502, Acc=81.12%
Epoch 13 (C10_S1): Loss=0.5198, Acc=82.35%
Epoch 14 (C10_S1): Loss=0.5024, Acc=82.69%
Epoch 15 (C10_S1): Loss=0.4802, Acc=83.61%
Epoch 16 (C10_S1): Loss=0.4617, Acc=84.13%
Epoch 17 (C10_S1): Loss=0.4410, Acc=84.95%
Epoch 18 (C10_S1): Loss=0.4274, Acc=85.30%
Epoch 19 (C10_S1): Loss=0.4020, Acc=86.31%
Epoch 20 (C10_S1): Loss=0.3938, Acc=86.52%
Epoch 21 (C10_S1): Loss=0.3794, Acc=86.94%
Epoch 22 (C10_S1): Loss=0.3658, Acc=87.44%
Epoch 23 (C10_S1): Loss=0.3496, Acc=88.09%
Epoch 24 (C10_S1): L

C100_S2_accuracy,▁▄▆▇████████████████████████████████████
C100_S2_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
C10_S1_accuracy,▁▃▅▅▆▆▆▆▇▇▇▇▇▇▇█████████████████████████
C10_S1_loss,█▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇████
C100_S2_accuracy,99.134
C100_S2_loss,0.03157
C10_S1_accuracy,99.388
C10_S1_loss,0.02239
epoch,200



Q4 finished. Two sequential runs logged to W&B.
